In [3]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Activation

In [90]:
def preprocess(file):
    df = pd.read_csv(file)
    df['Date'] = pd.to_datetime(df['Date'])
    df = df.sort_values(by='Date', ascending=True).reset_index(drop=True)
    df = df.rename(columns={'Price': 'Close'})
    df['Close-1'] = df['Close'].shift(1)
    df['z_open'] = df['Open']/df['Close'] - 1
    df['z_high'] = df['High']/df['Close'] - 1
    df['z_low'] = df['Low']/df['Close'] - 1
    df['z_close'] = df['Close']/df['Close-1'] - 1
    try:
        df['Adj Close'] is None
    except:
        df['Adj Close'] = df['Close']

    df['adj_close-1'] = df['Adj Close'].shift(1)
    # 수정주가 관련
    df['adj_close'] = df['Adj Close']/df['adj_close-1'] -1
    df['z_adj_close5'] = df['adj_close-1'].rolling(5).sum()/5*df['adj_close'] - 1
    df['z_adj_close10'] = df['adj_close-1'].rolling(10).sum()/10*df['adj_close'] - 1
    df['z_adj_close15'] = df['adj_close-1'].rolling(15).sum()/15*df['adj_close'] - 1
    df['z_adj_close10'] = df['adj_close-1'].rolling(20).sum()/20*df['adj_close'] - 1
    df['z_adj_close10'] = df['adj_close-1'].rolling(25).sum()/25*df['adj_close'] - 1
    df['z_adj_close10'] = df['adj_close-1'].rolling(30).sum()/30*df['adj_close'] - 1

    # df = df[['Open', 'High', 'Low', 'Close']]
    return df

In [91]:

df = preprocess('./data/k200.csv')
df

,Date,Close,Open,High,Low,Vol.,Change %,Close-1,z_open,z_high,z_low,z_close,Adj Close,adj_close-1,adj_close,z_adj_close5
0,2001-01-02,65.41,63.33,65.47,62.97,130.59M,3.25%,NaN,-0.031799,0.000917,-0.037303,NaN,65.41,NaN,NaN,NaN
1,2001-01-03,65.38,64.28,65.79,63.99,182.70M,-0.05%,65.41,-0.016825,0.006271,-0.021260,-0.000459,65.38,65.41,-0.000459,NaN
2,2001-01-04,70.14,69.19,71.37,69.19,265.50M,7.28%,65.38,-0.013544,0.017536,-0.013544,0.072805,70.14,65.38,0.072805,NaN
3,2001-01-05,73.15,70.32,73.23,69.74,352.41M,4.29%,70.14,-0.038688,0.001094,-0.046617,0.042914,73.15,70.14,0.042914,NaN
4,2001-01-08,73.83,72.12,74.01,71.98,378.30M,0.93%,73.15,-0.023161,0.002438,-0.025058,0.009296,73.83,73.15,0.009296,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,2021-03-19,412.91,416.70,416.71,410.59,189.02M,-1.00%,417.06,0.009179,0.009203,-0.005619,-0.009951,412.91,417.06,-0.009951,-5.136845
4996,2021-03-22,412.64,412.92,414.48,410.27,270.01M,-0.07%,412.91,0.000679,0.004459,-0.005744,-0.000654,412.64,412.91,-0.000654,-1.271428
4997,2021-03-23,408.71,412.89,416.15,408.40,234.50M,-0.95%,412.64,0.010227,0.018204,-0.000758,-0.009524,408.71,412.64,-0.009524,-4.950439
4998,2021-03-24,407.10,407.32,408.76,404.03,193.07M,-0.39%,408.71,0.000540,0.004078,-0.007541,-0.003939,407.10,408.71,-0.003939,-2.627159


In [83]:
# Tanh outputs between -1 and 1.

model = Sequential([
    Flatten(input_shape=(28, 28)),    # reshape 28 row * 28 column data to 28*28 rows
    Dense(256, activation='sigmoid'), # dense layer 1
    Dense(10, activation='softmax'),  # dense layer 3
])